# Подключение гугл диска

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Описание задачи

Данные были взяты с соревнования ods, которое закончилось пару месяцев назад (https://ods.ai/competitions/learning-analytics)

Я соедининил датафреймы train, comp_portrait, comp_marks и comp_disc и поставил задачу предсказания наличия у студента задолженности (по конкретной дисциплине в конкретный семестр) на основе имеющейся информации непосредственно в момент поступления в вуз. 

Таким образом, target (DEBT) является бинарным признаком (1 - есть задолженность, 0 - нет задолженностей), и мы будем решать задачу бинарной классификации 

Совокупность всех других признаков представляет собой описание попытки:

- SEMESTER - семестр получения оценки
- DISC_ID - UID дисциплины
- TYPE_NAME - форма отчётности
- GENDER - пол студента
- CITIZENSHIP - гражданство студента
- EXAM_TYPE - форма зачисления студента (ЕГЭ, олимпиада, ВИ - вступительные испытания)
- EXAM_SUBJECT_1 - первый экзамен ЕГЭ
- EXAM_SUBJECT_2 - второй экзамен ЕГЭ
- EXAM_SUBJECT_3 - третий экзамен ЕГЭ
- ADMITTED_EXAM_1 - баллы за 1 экзамен ЕГЭ
- ADMITTED_EXAM_2 - баллы за 2 экзамен ЕГЭ
- ADMITTED_EXAM_3 - баллы за 3 экзамен 
- ADMITTED_SUBJECT_PRIZE_LEVEL - уровень олимпиады (если есть)
- REGION_ID - номер региона студента

Также присутствуют бинаризованные признаки (каждый столбец представляет собой одно значение признака; 1 - значение соответствует объекту, 0 - не соответствует):
- MAIN_PLAN - учебный план
- PRED_ID - UID преподавателя
- DISC_DEP - факультет-организатор дисциплины
- CHOICE - выборность дисциплины

# Импорт библиотек

In [3]:
!pip install Catboost
!pip install optuna

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams
import yaml
import json

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, \
    recall_score, f1_score, log_loss, auc, classification_report, confusion_matrix, \
    precision_recall_curve, roc_curve

from catboost import CatBoostClassifier, Pool

import optuna

import warnings

warnings.filterwarnings("ignore")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 6.5 MB/s 
     |████████████████████████████████| 209 kB 55.5 MB/s 
     |████████████████████████████████| 81 kB 11.2 MB/s 
     |████████████████████████████████| 78 kB 8.1 MB/s 
     |████████████████████████████████| 50 kB 6.0 MB/s 
     |████████████████████████████████| 112 kB 52.0 MB/s 
     |████████████████████████████████| 147 kB 56.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=17f01736740392743d78d4341a65fdacbf75d59552ef831f6091111e19347f57
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


# Загрузка конфигурационного файла

Загрузим файл, в котором лежат все необходимые константы, пути и т.д. Это нужно для того, чтобы в случае каких-либо изменений можно было поменять одну строку в файле, а не искать по всему проекту места, в которых используется старое значение

In [ ]:
config_path = '/content/drive/MyDrive/params_.yml'
config = yaml.load(open(config_path), Loader=yaml.FullLoader)
preproc = config['preprocessing']
training = config['training']

# Считывание данных

Считаем обучающий датасет

In [ ]:
df = pd.read_parquet(preproc['df_path'])
df.drop(columns=['mean_score'], inplace=True)
df.head()

In [ ]:
df.columns

In [ ]:
df.to_parquet('/content/drive/MyDrive/df.parquet.gzip')

In [ ]:
df.shape

# Преобразование типов

Преобразуем категориальные столбцы в соответствующий тип данных

In [ ]:
def transform_types(data: pd.DataFrame, change_type_columns: dict) -> pd.DataFrame:
    """
    Преобразование признаков в заданный тип данных
    :param data: датасет
    :param change_type_columns: словарь с признаками и типами данных
    : return: преобразованный датасет
    """
    return data.astype(change_type_columns, errors="raise")

In [ ]:
df = transform_types(data=df, change_type_columns=preproc['change_type_columns'])

# Сохранение уникальных значений признаков

Сохраним списки уникальных значений для каждого признака, которые затем будем использовать для подачи через UI

In [ ]:
def save_unique_train_data(data: pd.DataFrame,
                           drop_columns: list,
                           target_column:str,
                           unique_values_path: str) -> None:
    """
    Сохранение словаря с признаками и уникальными значениями
    :param drop_columns: список с признаками для удаления
    :param data: датасет
    :param target_column: целевая переменная
    :param unique_values_path: путь до файла со словарём
    :return: None
    """

    unique_df = data.drop(
        columns=drop_columns + [target_column], axis=1, errors="ignore"
    )

    dict_unique = {key: list(unique_df[key].unique()) for key in unique_df.columns}  
    print(dict_unique)
    with open(unique_values_path, "w") as file:
      json.dump(dict_unique, file)

In [ ]:
save_unique_train_data(
    data=df[preproc['not_binary_columns']],
    drop_columns=[],
    target_column=preproc['target_column'],
    unique_values_path=preproc['unique_values_path']
)

In [ ]:
df.info()

# Exploratory data analysis

## Основные статистики

Посмотрим описательные статистики по float столбцам

In [ ]:
df.describe(include=["float64"])

Посмотрим описательные статистики по category столбцам

In [ ]:
df.describe(include="category")

## Target

Посмотрим распределение целевой переменной

In [ ]:
def plot_text(ax):
    """
    Добавление подписи процентов на график barplot
    :param ax: ось
    :return: None
    """
    for p in ax.patches:
        percentage = '{:.1f}%'.format(p.get_height())
        ax.annotate(
            percentage,  # текст
            # координата xy
            (p.get_x() + p.get_width() / 2., p.get_height()),
            # центрирование
            ha='center',
            va='center',
            xytext=(0, 10),
            # точка смещения относительно координаты
            textcoords='offset points',
            fontsize=14)

In [ ]:
def barplot(data: pd.DataFrame,
            col: str,
            title: str) -> None:
    """
    Построение графика распределения признака в виде столбчатой диаграммы
    :param data: датасет
    :param col: столбец, для которого которого хотим смотреть распределение
    :param title: заголовок
    :return: None 
    """
    rcParams['figure.figsize'] = 10, 8
    sns.color_palette("YlOrBr", as_cmap=True)

    # Датафрейм частот значений  
    norm_target = pd.DataFrame(df[col].value_counts(normalize=True).mul(100)\
                           .rename('percent')).reset_index()

    ax = sns.barplot(x='index', y='percent', data=norm_target, palette="flare")
    plt.title(title)
    plot_text(ax) 

In [ ]:
barplot(data=df,
        col=preproc['target_column'], 
        title='Распределение классов в разрезе target')

Видим сильный дисбаланс классов, это нужно будет учесть при обучении алгоритма

Придумаем ряд гипотез, которые помогут нам лучше понять данные, и сразу же их проверим

## 1 гипотеза

Доля задолженностей у мужчин больше, чем у женщин (иными словами, вероятность того, что случайно выбранный объект с gender='М' будет принадлежать целевому классу больше вероятности того, что случайно выбранный объект с gender='Ж' будет принадлежать целевому классу)

In [ ]:
def barplot_group(data: pd.DataFrame,
                  col: str,
                  col_group: str,
                  values_in_col_group: list,
                  title: str) -> None:
    """
    Построение графика распределения признака в виде ступенчатой диаграммы
    в разрезе другого бинарного признака
    :param data: датасет
    :param col: столбец, для которого хотим смотреть распределение
    :param col_group: столбец, в разрезе которого хотим смотреть распределение
    :values_in_col_group: список значений col_group
    :param title: заголовок
    :return: None 
    """
    rcParams['figure.figsize'] = 10, 8
    sns.color_palette("YlOrBr", as_cmap=True)
    
    # Датафрейм частот для каждого значения col_group
    dataframes_of_frequency = []

    for x in values_in_col_group:
        freq = df[df[col_group]==x][col]\
                .value_counts(normalize=True).rename('percent').reset_index()
        freq[col_group] = x
        dataframes_of_frequency.append(freq)

    # Общий датафрейм частот
    target_values = pd.concat(dataframes_of_frequency)
    target_values.rename(columns={"index": "target"}, inplace=True)
    target_values['percent']=target_values['percent']*100

    g = sns.catplot(x=col_group,
                    y='percent',
                    hue='target',
                    data=target_values,
                    kind='bar',
                    height=8,
                    palette=sns.color_palette(["indianred", "purple"]))
    plt.title(title)

    plot_text(g.ax);

In [ ]:
barplot_group(data=df,
              col=preproc['target_column'],
              col_group=preproc['gender_column'],
              values_in_col_group=preproc['values_in_gender_column'],
              title='Распределение target в разрезе пола')

Видим, что доля попыток, закончившихся задолженностью у мужчин почти в 2 раза больше, чем у женщин. Гипотеза подтвердилась, и можно сделать вывод, что пол является важным признаком в нашей задаче

## 2 гипотеза

Доля задолженностей у олимпиадников меньше, чем у тех, кто сдавал ЕГЭ или вступительные испытания.

Это выглядит здравым предположением, так как задачи олимпиад обычно сложнее, и победители должны потратить на них много времени, параллельно готовясь к ЕГЭ/ ВИ. Даже людей, просто участвующих в олимпиадах обычно не очень много, и сложно представить, что те, кто в них побеждают, могут иметь плохие оценки в вузе

In [ ]:
barplot_group(data=df,
              col=preproc['target_column'],
              col_group=preproc['exam_type_column'],
              values_in_col_group=preproc['values_in_exam_type_column'],
              title='Распределение target в разрезе типа экзамена')

Вторая гипотеза тоже подтвердилась - видим, что олимпиадники более успешны относительно других каст, хоть они и обогнали сдающих ЕГЭ всего на 0.4%. При этом, у людей, поступающих по вступительным испытаниям, доля неудач значительно выше

## 3 гипотеза

На экзаменах неудачи случаются чаще, чем на зачётах/диф. зачётах/при сдаче курсовых проектов

Экзамены в вузе, как правило, сложнее зачётов, и их нельзя сдавать много раз. Причём, в некоторых местах, если студент не сдал определённое кол-во зачётов, то к экзаменам его тоже не допустят. Аналогично, с курсовыми проектами - человек, не закрывший курсовую, автоматически получает 2 за экзамен. Поэтому логично предположить, что доля задолженностей выше для экзаменов, чем для других форм отчётности

In [ ]:
barplot_group(data=df,
              col=preproc['target_column'],
              col_group=preproc['type_name_column'],
              values_in_col_group=preproc['values_in_type_name_column'],
              title='Распределение target в разрезе типа отчётности')

Вторая гипотеза тоже подтвердилась - видим, что олимпиадники более успешны относительно других каст, хоть они и обогнали сдающих ЕГЭ всего на 0.4%. При этом, у людей, поступающих по вступительным испытаниям, доля неудач значительно выше

## 4 гипотеза

Вероятность наличия задолженности обратно пропорциональна среднему баллу поступления. Иными словами, в разрезе target распределения среднего балла отличаются, при этом распределение для строк, соответствующих успешной попытке, должно быть смещено вправо (т.е. в сторону максимального балла)

In [ ]:
def create_mean_column(data: pd.DataFrame,
                       cols: list) -> pd.DataFrame:
    """
    Добавление столбца средних по некоторому подмножеству признаков (столбцов)
    :param data: датасет
    :param cols: список столбцов, значения которых участвуют в вычислении среднего
    """
    # Столбец суммы по подмножеству признаков
    sum = 0
    for x in cols:
        sum += data[x]
   
    # Столбец средних
    data['mean'] = sum/len(cols)
    return data

In [ ]:
data = create_mean_column(data=df,
                   cols=preproc['addmited_exam_columns'])

In [ ]:
def displots_of_statistic(data: pd.DataFrame,
                         col: list,
                         col_group: str,
                         values_in_col_group: list,
                         title: str) -> None:
    """
    Построение графиков распределений значений одного столбца в разрезе значений
     другого столбца
    :param data: датасет
    :param col: столбец, по которому хотим смотреть распределение
    :param col_group: столбец, в разрезе которого хотим смотреть распределение
    :values_in_col_group: список значений col_group
    :param title: заголовок
    :return: None 
    """
    
    ''' Словарь, в котором значения являются подстолбцами столбца средних, соответствующие
    той или иной градации признака col_group. Ключи, по сути, являются идентификатором
    этой градации
    '''
    data_for_displot = {}
    for x in values_in_col_group:
        data_for_displot[col_group + ' ' + str(x)] = data[data[col_group] == x][col]

    sns.displot(
    data=data_for_displot,
    kind="kde",
    common_norm=False)

    plt.title(title)

In [ ]:
displots_of_statistic(data=df,
                     col='mean',
                     col_group=preproc['target_column'],
                     values_in_col_group=preproc['values_in_target_column'],
                     title='Распределение значений mean_score\n') 
  

In [ ]:
def boxplot(data: pd.DataFrame,
            x: str,
            y: str,
            title: str) -> None:
    """
    Построение графиков boxplot по столбцу в разрезе значений другого столбца
    :param data: датасет,
    :param x: столбец, в разрезе которого хотим строить график
    :param y: столбец, по которому хотим строить график
    :param title: заголовок
    :return: None
    """
    sns.boxplot(x=x, y=y, data=data)
    plt.title(title, fontsize=20)
    plt.ylabel(y, fontsize=14)
    plt.xlabel(x, fontsize=14)

In [ ]:
boxplot(data=df,
        x=preproc['target_column'],
        y='mean',
        title='Распределение значений mean_score\n')

In [ ]:
del df['mean']

Гипотеза в принципе подтвердилась - по первому графику видно, что синее распределение смещено вправо. Это значит, что попытки, заканчивающиеся задолженностью, чаще приходятся на людей, имеющих меньшие баллы при поступлении. Боксплот также демонстрирует смещение статистик, хоть и не очень сильное

# Обучение бейзлайна Catboost

In [ ]:
def get_metrics(y_test: list,
                y_pred: list,
                y_score: list,
                name: str) -> pd.DataFrame:
    """
    Подсчёт метрик бинарной классификации
    :param y_test: реальные метки классов
    :parsm y_pred: предсказание алгоритма
    :param y_score: вероятность того, что объект относится к целевому классу
    """
    df_metrics = pd.DataFrame()
    
    df_metrics['model'] = [name]

    df_metrics['Accuracy'] = [accuracy_score(y_test, y_pred)]
    df_metrics['ROC_AUC'] = [roc_auc_score(y_test, y_score[:,1])]
    df_metrics['Precision'] = [precision_score(y_test, y_pred)]
    df_metrics['Recall'] = [recall_score(y_test, y_pred)]
    df_metrics['f1'] = [f1_score(y_test, y_pred)]
    df_metrics['Logloss'] = [log_loss(y_test, y_score)]
    
    return df_metrics

Разобьём данные на train/test

In [ ]:
X = df.drop(training['target_column'], axis=1)
y = df[training['target_column']]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify=y,
                                                    shuffle=True,
                                                    test_size=training['test_size'],
                                                    random_state=training['random_state'])

Разобьём train на train_ и val_, чтобы использовать валидационное множество для проверки после построения каждого нового дерева

In [ ]:
X_train_, X_val, y_train_, y_val = train_test_split(X_train,
                                                    y_train,
                                                    stratify=y_train,
                                                    shuffle=True,
                                                    test_size=training['test_size'],
                                                    random_state=training['random_state'])
eval_set = [(X_val, y_val)]

In [ ]:
scale_pos_weight =  df[df[training['target_column']] == 0
                       ].shape[0] / df[df[training['target_column']] == 1].shape[0]

catboost = CatBoostClassifier(random_state=training['random_state'],
                              cat_features=training['category_features'],
                              scale_pos_weight=scale_pos_weight)
catboost.fit(X_train_,
        y_train_,
        eval_set=eval_set,
        early_stopping_rounds=training['early_stopping_round'])

Взглянем на метрики

In [ ]:
y_pred = catboost.predict(X_test)
y_pred_prob = catboost.predict_proba(X_test)
metrics = get_metrics(y_test, y_pred, y_pred_prob, name='Catboost')

y_pred_train = catboost.predict(X_train_)
y_pred_prob_train = catboost.predict_proba(X_train_)
metrics = metrics.append(
    get_metrics(y_train_, y_pred_train, y_pred_prob_train, name='Catboost_train'))

metrics

# Подбор гиперпараметров Catboost

In [ ]:
def f1_metric(labels, scores):
    """
    Реализация f1-метрики для подачи в Catboost
    :param labels: истинные метки классов
    :param scores: вероятности того, что объект принадлежит целевому классу
    """
    pred = np.round(scores)
    return 'f1', f1_score(labels, pred), True

In [ ]:
def objective_lgb(trial, X, y, N_FOLDS, random_state, cat_feat):
    params = {
        "n_estimators":
        trial.suggest_categorical("n_estimators", [training['n_estimators']]),
        "learning_rate":
        trial.suggest_categorical("learning_rate", [training['learning_rate']]),
        "max_depth":
        trial.suggest_int("max_depth", 3, 12),
        "l2_leaf_reg":
        trial.suggest_uniform("l2_leaf_reg", 1e-5, 1e2),
        "bootstrap_type":
        trial.suggest_categorical("bootstrap_type",
                                  ["Bayesian", "Bernoulli", "MVS", "No"]),
        "border_count":
        trial.suggest_categorical('border_count', [128, 254]),
        "grow_policy":
        trial.suggest_categorical('grow_policy',
                                  ["SymmetricTree", "Depthwise", "Lossguide"]),
        "auto_class_weights":
        trial.suggest_categorical("auto_class_weights",
                                  ["None", "Balanced", "SqrtBalanced"]),
        
        "cat_features":
        trial.suggest_categorical("cat_features", [training['category_features']]),
        "loss_function":
        trial.suggest_categorical("loss_function", ["Logloss"]),
        "use_best_model":
        trial.suggest_categorical("use_best_model", [True]),
        "random_state":
        training['random_state']
    }

    if params["bootstrap_type"] == "Bayesian":
        params["bagging_temperature"] = trial.suggest_float(
            "bagging_temperature", 0, 10)
    elif params["bootstrap_type"] == "Bernoulli":
        params["subsample"] = trial.suggest_float("subsample",
                                                  0.1,
                                                  1,
                                                  log=True)

    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=training['random_state'])

    cv_predicts = np.empty(N_FOLDS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        train_data = Pool(data=X_train, label=y_train, cat_features=cat_feat)
        eval_data = Pool(data=X_test, label=y_test, cat_features=cat_feat)

        model = CatBoostClassifier(**params)
        model.fit(train_data,
                  eval_set=eval_data,
                  early_stopping_rounds=training['early_stopping_round'],
                  verbose=0)
        
        preds = model.predict(X_test)
        cv_predicts[idx] = f1_score(y_test, preds)

    return np.mean(cv_predicts)

In [ ]:
study_cat = optuna.create_study(direction="maximize", study_name="Catboost")
func = lambda trial: objective_lgb(trial,
                                   X_train,
                                   y_train,
                                   N_FOLDS=training['n_folds'],
                                   random_state=training['random_state'],
                                   cat_feat=training['category_features'])

study_cat.optimize(func, n_trials=20, show_progress_bar=True)

In [ ]:
study.best_params